In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv
import signal
import sys

# Update ChromeDriver path
service = Service(r'C:\Users\sshak\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')  # Update the path
# Update this path
driver = webdriver.Chrome(service=service)

# URL to start scraping
url = "https://unsplash.com/s/photos/africa-people?license=free"

# Initialize CSV file to store links
output_csv = 'unsplash.csv'
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Download URL'])  # Write CSV header

# Set to store unique links
seen_links = set()

# Function to extract new download links
def extract_new_download_links(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    new_links = []
    for a_tag in soup.find_all('a', itemprop="contentUrl", class_="zNNw1", href=True):
        link = a_tag['href']
        if link not in seen_links:  # Add only new links
            new_links.append(link)
            seen_links.add(link)
    return new_links

# Function to save links to CSV
def save_links_to_csv(new_links):
    with open(output_csv, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for link in new_links:
            writer.writerow([link])
            print(f"Saved link: {link}")

# Function to handle manual interruption (Ctrl+C)
def handle_exit(signum, frame):
    print("\nManual stop detected. Saving progress...")
    driver.quit()  # Close the browser
    sys.exit(0)

# Register the signal handler
signal.signal(signal.SIGINT, handle_exit)

# Start scraping
try:
    driver.get(url)
    time.sleep(5)  # Allow the page to load initially

    print("Scrolling and extracting links...")
    while True:
        # Scroll down
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(1)  # Pause to allow content to load

        # Extract new links
        page_source = driver.page_source
        new_links = extract_new_download_links(page_source)

        # Save new links to CSV
        if new_links:
            save_links_to_csv(new_links)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    driver.quit()
    print(f"Scraping completed. Links are saved in '{output_csv}'.")


Scrolling and extracting links...
Saved link: /photos/man-in-white-shirt-looking-into-right-nWdhpUrXxYY
Saved link: /photos/selective-focus-of-man-smiling-during-daytime-2EGNqazbAMk
Saved link: /photos/woman-smiling-beside-red-wall-LWkFHEGpleE
Saved link: /photos/portrait-photography-of-man-95UF6LXe-Lo
Saved link: /photos/grayscale-photography-of-kid-4U1x6459Q-s
Saved link: /photos/children-sitting-on-chairs-inside-classroom-jEEYZsaxbH4
Saved link: /photos/people-dancing-during-daytime-9cx4-QowgLc
Saved link: /photos/three-women-carrying-basin-while-walking-barefoot-CG6Gd__QIOY
Saved link: /photos/woman-with-green-and-white-flower-on-ear-h1lA3N5wb8M
Saved link: /photos/woman-carrying-baby-on-back-yrzBgqapG1I
Saved link: /photos/smiling-boy-sitting-while-holding-soccer-ball-at-daytime-YEK-1BZ7L1s
Saved link: /photos/smiling-man-standing-between-brown-concrete-buildings-at-daytime-UiVe5QvOhao
Saved link: /photos/man-holding-drum-bhqsqkCowSk
Saved link: /photos/man-in-red-tank-top-standin

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import csv
import time


# Setup Selenium WebDriver
service = Service(r'C:\Users\sshak\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')  # Update the path

driver = webdriver.Chrome(service=service)


# Open the Unsplash URL
url = 'https://unsplash.com/s/photos/africa-people?license=free'
driver.get(url)

# Wait for the page to load completely
time.sleep(1)  # You can adjust the sleep time depending on the speed of the page loading

# Find the first image on the page (it might be inside a specific container or tag)
first_image = driver.find_element(By.XPATH, '//img[@srcset]')  # You can adjust this XPath based on the structure of the page

# Scroll to the image to ensure it's in view
# actions = ActionChains(driver)
# actions.move_to_element(first_image).perform()

# Click on the first image
first_image.click()

# Wait a few seconds to ensure the image is opened
# time.sleep(2)
# Open CSV file in write mode and set up CSV writer
with open('download_links.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Download URL"])  # Write header

    # Loop to extract and save download URLs
    for _ in range(10):  # Modify the range to loop as many times as needed
        try:
            # Find the download button using its data-testid attribute
            download_button = driver.find_element(By.XPATH, '//a[@data-testid="non-sponsored-photo-download-button"]')

            # Get the download link from the button's href attribute
            download_url = download_button.get_attribute('href')
            print("Download URL:", download_url)

            # Write the download URL into the CSV file
            writer.writerow([download_url])

            # Simulate pressing the RIGHT ARROW key to go to the next image
            actions = ActionChains(driver)
            actions.send_keys(Keys.RIGHT).perform()

            # Wait for the page to load the next image (adjust time as needed)
            time.sleep(3)

        except Exception as e:
            print(f"Error: {e}")
            break  # Exit the loop if there's an error (like no more images)

# Close the WebDriver
driver.quit()

C:\Users\sshak\AppData\Local\Temp\ipykernel_24356\2764079837.py:46: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  download_url = download_button.get_attribute('href')


Download URL: https://unsplash.com/photos/nWdhpUrXxYY/download?ixid=M3wxMjA3fDB8MXxzZWFyY2h8MXx8YWZyaWNhJTIwcGVvcGxlfGVufDB8fHx8MTczMzc2NDE3M3wy&force=true
Error: Message: element click intercepted: Element <img alt="man in white shirt looking into right" sizes="(min-width: 1351px) 416px, (min-width: 992px) calc(calc(100vw - 88px) / 3), (min-width: 768px) calc(calc(100vw - 64px) / 2), 100vw" srcset="https://images.unsplash.com/photo-1547037579-f0fc020ac3be?w=100&amp;auto=format&amp;fit=crop&amp;q=60&amp;ixlib=rb-4.0.3&amp;ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8YWZyaWNhJTIwcGVvcGxlfGVufDB8fDB8fHwy 100w, https://images.unsplash.com/photo-1547037579-f0fc020ac3be?w=200&amp;auto=format&amp;fit=crop&amp;q=60&amp;ixlib=rb-4.0.3&amp;ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8YWZyaWNhJTIwcGVvcGxlfGVufDB8fDB8fHwy 200w, https://images.unsplash.com/photo-1547037579-f0fc020ac3be?w=300&amp;auto=format&amp;fit=crop&amp;q=60&amp;ixlib=rb-4.0.3&amp;ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8YWZyaWNhJTIwcGVvcGxlfGVufDB8fDB8fHwy 

In [56]:
import csv
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Configure Selenium WebDriver
service = Service(r'C:\Users\sshak\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')  # Update with the correct path to your chromedriver
driver = webdriver.Chrome(service=service)

# Input CSV file containing URLs
input_csv = 'download_links.csv'  # Update with your input CSV file name
output_folder = 'indians'  # Folder to save downloaded images

# Ensure output folder exists
import os

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to extract and download an image from a URL
def download_image_from_url(url, image_name):
    try:
        driver.get(url)
        driver.implicitly_wait(10)  # Wait for the page to load
        
        # Get the page source after it's fully loaded
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find the correct <source> tag: type="image/jpeg" and no media attribute
        source_tag = soup.find('source', type="image/jpeg", media=False)
        # if source_tag and source_tag.has_attr('srcset'):
        #     img_url = source_tag['srcset']
            
        #     # Download the image
        #     response = requests.get(img_url)
        #     if response.status_code == 200:
        #         # Save the image to a file
        #         file_path = os.path.join(output_folder, image_name)
        #         with open(file_path, 'wb') as file:
        #             file.write(response.content)
        #         # print(f"Image downloaded successfully: {file_path}")
        #     else:
        #         print(f"Failed to download image from {url}. Status code:", response.status_code)
        # else:
        #     print(f"No valid <source> tag with desired attributes found for URL: {url}")
    except Exception as e:
        print(f"Error processing {url}: {e}")

# Read the input CSV file and process each URL
with open(input_csv, mode='r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    for i, row in enumerate(reader, start=1):
        source_url = row[0]  # Assuming each row contains one URL
        image_name = f"{i}.jpg"  # Naming images sequentially
        print(f"Processing URL {i}: {source_url}")
        download_image_from_url(source_url, image_name)

# Close the Selenium WebDriver
driver.quit()

print("Image downloading completed.")


Processing URL 1: Download URL
Error processing Download URL: Message: invalid argument
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF71C5A6CB5+28821]
	(No symbol) [0x00007FF71C513840]
	(No symbol) [0x00007FF71C3B55B9]
	(No symbol) [0x00007FF71C3A3051]
	(No symbol) [0x00007FF71C3A12FD]
	(No symbol) [0x00007FF71C3A1B3C]
	(No symbol) [0x00007FF71C3B885A]
	(No symbol) [0x00007FF71C4501FE]
	(No symbol) [0x00007FF71C42F2FA]
	(No symbol) [0x00007FF71C44F412]
	(No symbol) [0x00007FF71C42F0A3]
	(No symbol) [0x00007FF71C3FA778]
	(No symbol) [0x00007FF71C3FB8E1]
	GetHandleVerifier [0x00007FF71C8DFCAD+3408013]
	GetHandleVerifier [0x00007FF71C8F741F+3504127]
	GetHandleVerifier [0x00007FF71C8EB5FD+3455453]
	GetHandleVerifier [0x00007FF71C66BDBB+835995]
	(No symbol) [0x00007FF71C51EB5F]
	(No symbol) [0x00007FF71C51A814]
	(No symbol) [0x00007FF71C51A9AD]
	(No symbol) [0x00007FF71C50A199]
	BaseThreadInitThunk [0x00007FFC6023E8D7+23]
	RtlUserThreadStart [0x00007FFC616

AssertionError: C:\path_to_files\deploy.prototxt does not exist!

In [ ]:
https://unsplash.com/photos/man-in-white-shirt-looking-into-right-nWdhpUrXxYY

SyntaxError: invalid syntax (2136811287.py, line 1)

In [ ]:
import cv2
import os
from mtcnn import MTCNN
import numpy as np

# Initialize face detector
detector = MTCNN()

def is_straight_face(landmarks):
    left_eye = landmarks['left_eye']
    right_eye = landmarks['right_eye']
    nose = landmarks['nose']

    # Calculate the angle between the eyes and nose
    eye_line = np.linalg.norm(np.array(right_eye) - np.array(left_eye))
    eye_to_nose_line = np.linalg.norm(np.array(nose) - ((np.array(left_eye) + np.array(right_eye)) / 2))
    
    # Adjust thresholds as needed
    return eye_to_nose_line / eye_line < 0.5

def is_clear_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var > 100  # Threshold for sharpness

def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        filepath = os.path.join(input_folder, filename)
        image = cv2.imread(filepath)
        if image is None:
            continue

        # Detect face
        results = detector.detect_faces(image)
        if results:
            face = results[0]
            if is_straight_face(face['keypoints']) and is_clear_image(image):
                # Save filtered image
                cv2.imwrite(os.path.join(output_folder, filename), image)

# Define paths
input_folder = r"C:\Users\sshak\Downloads\validation_2\400+"
output_folder = "path_to_output_folder"

process_images(input_folder, output_folder)


In [2]:
!pip install mtcnn


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.9 MB 8.2 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.9 MB 8.2 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.9 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 1.3/1.9 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 1.8 MB/s eta 0:00:00


In [1]:
import cv2
import dlib
import os
import numpy as np

input_folder = r"C:\Users\sshak\Downloads\validation_2\400+"
output_folder = "path_to_output_folder"
os.makedirs(output_folder, exist_ok=True)

# Load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to check blur
def is_clear(image, threshold=100.0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance > threshold

# Process images
for image_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, image_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Check face orientation (example: nose alignment with eyes)
        left_eye = landmarks.part(36).y
        right_eye = landmarks.part(45).y
        nose = landmarks.part(30).y

        if abs(left_eye - right_eye) < 10 and abs(nose - (left_eye + right_eye) / 2) < 10:
            face_img = img[face.top():face.bottom(), face.left():face.right()]
            
            # Check clarity
            if is_clear(face_img):
                save_path = os.path.join(output_folder, image_name)
                cv2.imwrite(save_path, face_img)

print("Filtered images saved to:", output_folder)


ModuleNotFoundError: No module named 'dlib'

In [22]:
import cv2
import os

def is_blurry(image, threshold=50):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance < threshold

input_folder = "new sharp_faces"

output_folder = "sharp_faces"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)
    image = cv2.imread(img_path)
    if image is None:
        continue
    
    if not is_blurry(image):  # Keep only sharp images
        cv2.imwrite(os.path.join(output_folder, filename), image)


In [2]:
import cv2
import os
import mediapipe as mp

def calculate_face_variance(image):
    """
    Detects the face region using MediaPipe and calculates the Laplacian variance for that region.
    """
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection()

    # Convert the image to RGB for MediaPipe
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect faces
    results = face_detection.process(rgb_image)

    if results.detections:
        # Assume the first detected face
        detection = results.detections[0]
        bboxC = detection.location_data.relative_bounding_box

        # Convert relative coordinates to absolute pixel values
        h, w, _ = image.shape
        x1 = int(bboxC.xmin * w)
        y1 = int(bboxC.ymin * h)
        x2 = int((bboxC.xmin + bboxC.width) * w)
        y2 = int((bboxC.ymin + bboxC.height) * h)

        # Ensure the bounding box is within the image dimensions
        x1, y1 = max(x1, 0), max(y1, 0)
        x2, y2 = min(x2, w), min(y2, h)

        # Crop the face region
        face_region = image[y1:y2, x1:x2]

        # Convert the face region to grayscale
        gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)

        # Calculate variance of the Laplacian
        variance = cv2.Laplacian(gray_face, cv2.CV_64F).var()

        return variance
    else:
        return None  # No face detected

def process_folder(input_folder, output_folder):
    """
    Process all images in the input folder, calculate face variance, and write it on each image.
    Saves the processed images to the output folder.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Check if it's an image
        if not (filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg")):
            continue

        # Read the image
        image = cv2.imread(file_path)
        if image is None:
            print(f"Skipping {filename}: Unable to read the image.")
            continue

        # Calculate face variance
        variance = calculate_face_variance(image)
        if variance is None:
            print(f"Skipping {filename}: No face detected.")
            continue

        # Define the position and font for the text
        text = f"Face Variance: {variance:.2f}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        bottom_left_corner = (10, image.shape[0] - 10)  # Bottom-left corner
        font_scale = 1
        color = (0, 255, 0)  # Green color
        thickness = 2

        # Put the text on the image
        cv2.putText(image, text, bottom_left_corner, font, font_scale, color, thickness)

        # Save the image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)
        print(f"Processed {filename} and saved to {output_path}")


input_folder ="sharp_faces"
output_folder = "varience sharp_faces" # Replace with your output folder path

# Run the function
process_folder(input_folder, output_folder)


Processed cropped_face_1001.png and saved to varience sharp_faces\cropped_face_1001.png
Processed cropped_face_1006.png and saved to varience sharp_faces\cropped_face_1006.png
Processed cropped_face_1016.png and saved to varience sharp_faces\cropped_face_1016.png
Processed cropped_face_10193.png and saved to varience sharp_faces\cropped_face_10193.png
Processed cropped_face_10249.png and saved to varience sharp_faces\cropped_face_10249.png
Processed cropped_face_10355.png and saved to varience sharp_faces\cropped_face_10355.png
Processed cropped_face_10356.png and saved to varience sharp_faces\cropped_face_10356.png
Processed cropped_face_1043.png and saved to varience sharp_faces\cropped_face_1043.png
Processed cropped_face_10494.png and saved to varience sharp_faces\cropped_face_10494.png
Processed cropped_face_1060.png and saved to varience sharp_faces\cropped_face_1060.png
Processed cropped_face_10760.png and saved to varience sharp_faces\cropped_face_10760.png
Processed cropped_fa

In [13]:
import cv2
import mediapipe as mp
import os
import shutil

# Initialize MediaPipe face detection and face mesh
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh


input_folder ="faces"
output_folder = "faces80"
os.makedirs(output_folder, exist_ok=True)


# Function to check if the face region is blurry using Laplacian variance
def is_blurry(face_region, threshold=100.0):  # Reduced the threshold value
    if face_region is None or face_region.size == 0:
        return True  # Return True for blurry if face region is empty
    
    # Convert face region to grayscale
    gray = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    # Compute the Laplacian of the face region and then compute the variance
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance < threshold  # Lower threshold makes it less strict

# Function to detect landmarks and calculate the quality score
def check_face_quality(image):
    # Convert the image to RGB (MediaPipe expects RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Initialize MediaPipe Face Detection and Face Mesh
    with mp_face_detection.FaceDetection(min_detection_confidence=0.9) as face_detection, \
         mp_face_mesh.FaceMesh(min_detection_confidence=0.9, min_tracking_confidence=0.9) as face_mesh:
        
        # Process the image to get face detection results
        face_detection_results = face_detection.process(image_rgb)
        
        if face_detection_results.detections:
            for detection in face_detection_results.detections:
                # Check the confidence score of the detected face
                confidence = detection.score[0]
                
                if confidence < 0.9:
                    continue  # Skip faces with low detection confidence
                
                # Get the bounding box of the face
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                face = image[y:y+h, x:x+w]
                
                # Ensure the face region is valid (non-empty)
                if face is None or face.size == 0:
                    continue  # Skip if face region is empty or invalid
                
                # Check if the face is blurry (face region only)
                if is_blurry(face):
                    continue  # Skip blurry faces
                
                # Detect landmarks
                results = face_mesh.process(image_rgb)
                
                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        total_landmarks = 468  # MediaPipe's FaceMesh detects 468 landmarks
                        detected_landmarks = 0

                        for landmark in face_landmarks.landmark:
                            if landmark.x > 0 and landmark.y > 0:  # Check if the landmark is visible
                                detected_landmarks += 1

                        # Calculate the percentage of detected landmarks
                        percentage_detected = (detected_landmarks / total_landmarks) * 100
                        
                        # Check if the face has sufficient landmarks detected (90%)
                        if percentage_detected >= 90:
                            return True  # Quality face detected
    return False  # No quality face detected

# Process each image in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    
    if os.path.isfile(file_path):
        image = cv2.imread(file_path)

        # Check if the face in the image is of good quality
        if check_face_quality(image):
            # Move image to output folder if the face is of good quality
            shutil.move(file_path, os.path.join(output_folder, filename))
            print(f"{filename} moved to output folder.")

print("Processing completed.")


1 (10121).png moved to output folder.
1 (10123).png moved to output folder.
1 (10132).png moved to output folder.
1 (10139).png moved to output folder.
1 (10141).png moved to output folder.
1 (10142).png moved to output folder.
1 (10144).png moved to output folder.
1 (10145).png moved to output folder.
1 (10151).png moved to output folder.
1 (10152).png moved to output folder.
1 (10156).png moved to output folder.
1 (10159).png moved to output folder.
1 (10160).png moved to output folder.
1 (10162).png moved to output folder.
1 (10165).png moved to output folder.
1 (10177).png moved to output folder.
1 (10179).png moved to output folder.
1 (10186).png moved to output folder.
1 (10187).png moved to output folder.
1 (10195).png moved to output folder.
1 (10196).png moved to output folder.
1 (10197).png moved to output folder.
1 (10201).png moved to output folder.
1 (10206).png moved to output folder.
1 (10210).png moved to output folder.
1 (10213).png moved to output folder.
1 (10219).pn